In [1]:
import sys;
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

import hetero_static_graph_temporal_signal as sgt 
import sgt_loader 
import asgrn 

In [2]:
salesdata = 'coffeedata_200upc_93store.csv'
nodes = 'Node_attr_coffee.csv' 
categoryname = 'coffee'

data = sgt.SalesDatasetLoader(salesdata,nodes,None)

In [3]:
H = 8
btr = []
bts = []
for i in range(len(data.gcndata)):
    dataset = data.gcndata[i].get_dataset(20,H)
    train_dataset, test_dataset = sgt.temporal_signal_split(dataset, train_ratio=0.8,H=H)
    itr = iter(train_dataset)
    btr += [next(itr) for j in range(train_dataset.snapshot_count)]
    its = iter(test_dataset)
    bts +=[next(its) for j in range(test_dataset.snapshot_count)]


In [4]:
#### model training
from random import seed 
seed(12345)

epochs= 200

PATH= './'+categoryname+'/'  
attr_levels = data._nodes.max(axis=0)+1

acc, loss_history =[], []

trloader = sgt_loader.DataLoader(btr, batch_size= 93, shuffle=True) 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

qq = torch.FloatTensor(np.array([[0.5,0.7,0.8,0.9,0.95]])).to(device)
#model = asgrn.seq2seq_quantile(node_features=14, window_length=20, horizon = H, hidden_state= 4,nquantiles=5, device=device).to(device)

model = asgrn.ASG_seq2seq_quantile_onehot(node_features=14, window_length=20, horizon = H, attr_levels = attr_levels, K=10,hidden_state= 4,nquantiles=5, device=device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
modelname= model.name + "_"+ categoryname

for epoch in tqdm(range(epochs)):

    model.train()
    for i, window in enumerate(trloader):
        window  = window.to(device)
        
        optimizer.zero_grad()
        y_pred,sl= model(window)

        scale_mask = window['node'].scale > 0
        mask = window['node'].mask *scale_mask.unsqueeze(1)            
        err = mask.unsqueeze(1)*(-y_pred *(window['node'].scale.unsqueeze(1).unsqueeze(2)) + window['node'].y.unsqueeze(1))           
        
        loss = torch.max(err *qq.unsqueeze(2),err*(qq.unsqueeze(2)-1)).mean()      
        
        loss.backward()
        optimizer.step()



   

Using cuda device


100%|██████████| 200/200 [30:37<00:00,  9.19s/it]


In [5]:
#### model forecasting
total_loss = 0
count = 0
agg_err = 0
predictions = []
acc = []
tsloader = sgt_loader.DataLoader(bts, batch_size= test_dataset.snapshot_count, shuffle=False)
with torch.no_grad():
    for i, window in enumerate(tsloader):
        window  = window.to(device)
        y_pred,_ = model(window)        
        scale_mask = window['node'].scale > 0
        mask = window['node'].mask *scale_mask.unsqueeze(1)        
        err = mask.unsqueeze(1)*(-y_pred *(window['node'].scale.unsqueeze(1).unsqueeze(2)) + window['node'].y.unsqueeze(1))       
        
        predictions.append(
            { 'tdx':window.tdx.cpu().numpy(),
                'mask': mask.cpu().numpy(),
                'preds': (y_pred*window['node'].scale.unsqueeze(1).unsqueeze(2)).cpu().numpy(),
                'y_true': window['node'].y.cpu().numpy(),
                'z_true': window['node'].z.cpu().numpy(),
                'store':i,
                })
        
        if mask.sum()>0 :
            total_loss += torch.max(err *qq.unsqueeze(2),err*(qq.unsqueeze(2)-1)).sum(axis=0)
            count += torch.sum(mask.unsqueeze(1),axis=0)
acc=(total_loss/count).cpu().numpy()

In [7]:
acc

array([[0.70747524, 0.7123235 , 0.71418947, 0.7261891 , 0.73500264,
        0.73769593, 0.75923806, 0.76827395],
       [0.68459094, 0.68880606, 0.6909569 , 0.7059837 , 0.7155797 ,
        0.7181954 , 0.7402955 , 0.7486615 ],
       [0.5895566 , 0.5927703 , 0.5939641 , 0.60963774, 0.6192363 ,
        0.62105733, 0.64183736, 0.64835954],
       [0.40786618, 0.40927   , 0.40948632, 0.4231889 , 0.4324725 ,
        0.43387803, 0.45064875, 0.4535854 ],
       [0.26225996, 0.2638931 , 0.26326445, 0.2733063 , 0.28108665,
        0.28109604, 0.293356  , 0.2944867 ]], dtype=float32)